In [9]:
import os
import random
import pandas as pd
import time   as time
from bs4          import BeautifulSoup
from datetime     import datetime
from urllib.parse import urlparse, urljoin
from config       import new_chromedriver_path, user_agents, path, connection_string
from web_scraper  import WebScraper
from vpn          import VPNChanger
from selenium.webdriver.support.ui  import WebDriverWait
from selenium.webdriver.support     import expected_conditions as EC
from selenium.webdriver.common.by   import By
from selenium.webdriver.common.keys import Keys
# from azure.storage.blob import BlobServiceClient

### Capturing links from Milano regions

In [10]:
# scraper = WebScraper('chromedriver-win64/chromedriver.exe',user_agents, path)
# driver = scraper.header_configuration(headless=False)
# driver.get(f'https://www.immobiliare.it/pt/vendita-case/milano/')
# html = BeautifulSoup(driver.page_source, 'html.parser')

# neighborhood_html = html.find_all('a', class_='in-breadcrumbLink__dropdownLink')

# neighborhood_list = []
# for n in neighborhood_html:
#     data = {
#         'neighborhood': n.text.strip(),
#         'link': n.get('href')
#     }

#     neighborhood_list.append(data)
    
# df_neighborhood = pd.DataFrame(neighborhood_list)

# df_neighborhood.to_csv('neighborhood.csv', index=False, encoding='utf-8-sig')

In [11]:
# complete_neighborhood_list = []

# for i in range(len(df_neighborhood)):
#     driver.get(df_neighborhood['link'].iloc[i]) 

#     try:
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.nd-list.nd-list--arrow.in-breadcrumb'))
#         )
#     except Exception as e:
#         print(f"Erro ao carregar a página: {e}")
#         continue

#     soup = BeautifulSoup(driver.page_source, 'html.parser')
#     ul_neighborhood = soup.find('ul', class_='nd-list nd-list--arrow in-breadcrumb')
    
#     if ul_neighborhood:
#         li = ul_neighborhood.find_all('li', class_='nd-list__item')
#         drill_down_list = li[-1].find_all('a', class_='in-breadcrumbLink__dropdownLink')

#         for d in drill_down_list:
#             data = {
#                 'neighborhood': df_neighborhood['neighborhood'].iloc[i],
#                 'drill_down': d.text.strip(),
#                 'link': d.get('href')
#             }

#             complete_neighborhood_list.append(data)

#     # Aguarde 1 segundo entre as requisições
#     time.sleep(1)

# df_complete_neighborhood = pd.DataFrame(complete_neighborhood_list)
# df_complete_neighborhood.to_csv('complete_neighborhood.csv', index=False, encoding='utf-8-sig')

### Ad scrap by region

In [12]:
scraper = WebScraper('chromedriver-win64/chromedriver.exe',user_agents, path)
driver = scraper.header_configuration(headless=False)

2024-10-11 23:14:24.004 | DEBUG    | web_scraper:ensure_cache_path:39 - Verificando se o caminho do cache existe: info_cache


In [13]:
df_complete_neighborhood = pd.read_csv("complete_neighborhood.csv")
df_complete_neighborhood.link = df_complete_neighborhood.link.str.replace('pt/','')

In [14]:
df_complete_neighborhood.head(5)

,neighborhood,drill_down,link
0,"Corvetto, Rogoredo",Corvetto,https://www.immobiliare.it/vendita-case/milano...
1,"Corvetto, Rogoredo",Rogoredo,https://www.immobiliare.it/vendita-case/milano...
2,"Precotto, Turro",Gorla,https://www.immobiliare.it/vendita-case/milano...
3,"Precotto, Turro",Precotto,https://www.immobiliare.it/vendita-case/milano...
4,"Precotto, Turro",Turro,https://www.immobiliare.it/vendita-case/milano...


In [15]:
output_folder = 'output'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [16]:
# driver.get(df_complete_neighborhood.link.iloc[0])

# html = BeautifulSoup(driver.page_source, 'html.parser')
# ul = html.find('ul', class_='nd-list in-searchLayoutList ls-results')
# li = ul.find_all('li', class_='nd-list__item in-searchLayoutListItem')

In [17]:
data_list = []

r = 1  # Contador de região
# Iterando por todos os links do df_complete_neighborhood
for idx in range(len(df_complete_neighborhood)):
    driver.get(df_complete_neighborhood.link.iloc[idx])

    ### Numerar última página
    html = BeautifulSoup(driver.page_source, 'html.parser')
    ul = html.find('ul', class_='nd-list in-searchLayoutList ls-results') if html else None
    if not ul:
        continue
    li = ul.find_all('li', class_='nd-list__item in-searchLayoutListItem') if ul else None
    if not li:
        continue
    div_pagination__list = html.find_all('div', class_='in-pagination__list')

    if div_pagination__list:
        for div in div_pagination__list:
            # Procura o número da última página
            pagination_items = div.find_all('div', class_='nd-button nd-button--ghost is-disabled in-pagination__item is-mobileHidden')

            if pagination_items:
                last_page = int(pagination_items[-1].text.strip())
            else:
                last_page = int(div_pagination__list[0].find_all('a')[-1].text.strip())
    else:
        last_page = 1  # Se não houver paginação, considera apenas a primeira página
    ######### Fim da Numeração

    
    print(f'Iniciando scraping do link {df_complete_neighborhood.drill_down.iloc[idx]}')
    print(f'{r}/{len(df_complete_neighborhood)}')
    print('#################################')
    
    intermediary_list = []

    #### Início do scraping dessa região
    for i in range(1, last_page + 1):  # Itera até a última página (incluindo last_page)
        if i == 1:
            print(f'Página {i}/{last_page} do link {df_complete_neighborhood.link.iloc[idx]}')
        else:
            print(f'Página {i}/{last_page} do link {next_page}')

        # Carrega o HTML e faz o parsing
        html = BeautifulSoup(driver.page_source, 'html.parser')
        ul = html.find('ul', class_='nd-list in-searchLayoutList ls-results') if html else None
        if not ul:
            break
        li = ul.find_all('li', class_='nd-list__item in-searchLayoutListItem') if ul else None
        if not li:
            break

        for l in li:
            div_pagination_control = html.find_all('div', class_='in-pagination__control')

            # Captura os dados do projeto
            project_id = l.find('div', class_='in-listingCard').get('id')
            link = l.find('a').get('href')
            ads_title = l.find('a').get('title')

            # Verifica se há múltiplas unidades (div.nd-stripe__item)
            listing_card_units = l.find('div', class_='nd-stripe__item in-listingCardUnits__item')
            if listing_card_units:
                for card in l.find_all('div', class_='nd-stripe__item in-listingCardUnits__item'):
                    image = card.find('img')
                    image = image.get('src') if image else None
                    title = card.find('div', class_='in-listingCardUnit__title')
                    title = title.text.strip() if title else None
                    price = card.find('span', class_='in-listingCardUnit__price')
                    price = price.text.strip().replace('€', '').replace('\xa0', '').replace('.', '').strip() if price else None

                    surface, bathrooms, floor, rooms = None, None, None, None
                    features = card.find_all('div', class_='in-listingFeatures__item')
                    if features:
                        for f in features:
                            if 'data-type' in f.attrs:
                                if f['data-type'] == 'surface':
                                    surface = f.text.strip()
                                elif f['data-type'] == 'bathrooms':
                                    bathrooms = f.text.strip()
                                elif f['data-type'] == 'floor':
                                    floor = f.text.strip()
                                elif f['data-type'] == 'rooms':
                                    rooms = f.text.strip()

                    data = {
                        'project_id': project_id,
                        'ads_title': ads_title,
                        'link': link,
                        'image': image,
                        'title': title,
                        'price': price,
                        'surface': surface,
                        'bathrooms': bathrooms,
                        'floor': floor,
                        'rooms': rooms,
                        'page': i,
                        'neighborhood_link': df_complete_neighborhood.link.iloc[idx],
                        'neighborhood': df_complete_neighborhood.neighborhood.iloc[idx],
                        'drill_down': df_complete_neighborhood.drill_down.iloc[idx]
                    }
                    data_list.append(data)
                    intermediary_list.append(data)

            else:
                # Captura os dados do projeto se não houver múltiplas unidades
                image = l.find('figure', class_='nd-figure nd-ratio nd-ratio--standard in-listingPhotos in-listingCardPropertyMediaMosaic__item')
                image = image.img.get('src') if image else None
                title = None
                price = l.find('div', class_='in-listingCardPrice')
                price = price.text.strip().replace('€', '').replace('\xa0', '').replace('.', '').strip() if price else None
                surface, bathrooms, floor, rooms = None, None, None, None
                features = l.find_all('div', class_='in-listingCardFeatureList__item')[:4]

                if features:
                    for f in features:
                        use_tag = f.find('use')
                        if use_tag and 'xlink:href' in use_tag.attrs:
                            href = use_tag.get('xlink:href')
                            if href == '#size':
                                surface = f.text.strip()
                            elif href == '#bath':
                                bathrooms = f.text.strip()
                            elif href == '#stairs':
                                floor = f.text.strip()
                            elif href == '#planimetry':
                                rooms = f.text.strip()

                data = {
                    'project_id': project_id,
                    'ads_title': ads_title,
                    'link': link,
                    'image': image,
                    'title': title,
                    'price': price,
                    'surface': surface,
                    'bathrooms': bathrooms,
                    'floor': floor,
                    'rooms': rooms,
                    'page': i,
                    'neighborhood_link': df_complete_neighborhood.link.iloc[idx],
                    'neighborhood': df_complete_neighborhood.neighborhood.iloc[idx],
                    'drill_down': df_complete_neighborhood.drill_down.iloc[idx]
                }
                data_list.append(data)
                intermediary_list.append(data)

        # Verifica se estamos na última página antes de tentar a próxima
        try:
            if i < last_page:
                next_page = fr'{df_complete_neighborhood.link.iloc[idx]}?pag={i+1}'
                time.sleep(2)  # Pausa para evitar sobrecarregar o servidor
                driver.get(next_page)
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.nd-list.in-searchLayoutList.ls-results'))
                )
            else:
                break
        except:
            break
    print('FINALIZADO\n#################################')
    r += 1
    # Salva intermediários a cada página
    df_intermediate = pd.DataFrame(intermediary_list)
    df_intermediate.to_csv(f'{output_folder}/intermediate_microzone_{df_complete_neighborhood.drill_down.iloc[idx]}.csv', index=False, encoding='utf-8-sig')

# Gera o DataFrame principal e salva
df_complete = pd.DataFrame(data_list)
df_complete.to_csv(f'{output_folder}/complete_neighborhood_data.csv', index=False, encoding='utf-8-sig')

print("Scraping finalizado e dados salvos com sucesso!")

Iniciando scraping do link Corvetto
1/144
#################################
Página 1/15 do link https://www.immobiliare.it/vendita-case/milano/corvetto/
Página 2/15 do link https://www.immobiliare.it/vendita-case/milano/corvetto/?pag=2
Página 3/15 do link https://www.immobiliare.it/vendita-case/milano/corvetto/?pag=3
Página 4/15 do link https://www.immobiliare.it/vendita-case/milano/corvetto/?pag=4
Página 5/15 do link https://www.immobiliare.it/vendita-case/milano/corvetto/?pag=5
Página 6/15 do link https://www.immobiliare.it/vendita-case/milano/corvetto/?pag=6
Página 7/15 do link https://www.immobiliare.it/vendita-case/milano/corvetto/?pag=7
Página 8/15 do link https://www.immobiliare.it/vendita-case/milano/corvetto/?pag=8
Página 9/15 do link https://www.immobiliare.it/vendita-case/milano/corvetto/?pag=9
Página 10/15 do link https://www.immobiliare.it/vendita-case/milano/corvetto/?pag=10
Página 11/15 do link https://www.immobiliare.it/vendita-case/milano/corvetto/?pag=11
Página 12/15 

### Subir GCP

In [18]:
from datetime               import datetime
from google.cloud           import bigquery
from google.oauth2          import service_account
from google.cloud.bigquery  import Client

In [19]:
credentials = service_account.Credentials.from_service_account_file('methodius-advisory-86691f61031e.json')
client = bigquery.Client(credentials=credentials, project='methodius-advisory')

In [20]:
def get_data(sql_file, var):  
    if var == True:
        dados = client.query(sql_file)
        result = dados.result()
        df = result.to_dataframe()
        return df
    else:
        with open(sql_file, 'r') as f:
            sql = f.read()
        dados = client.query(sql)
        result = dados.result()
        df = result.to_dataframe()
        return df

In [21]:
max_rows = 1048576

def save_to_multiple_sheets(df, writer, sheet_name_prefix):
    num_chunks = -(-len(df) // (max_rows - 1))
    for i in range(num_chunks):
        start_idx = i * (max_rows - 1)
        end_idx = (i + 1) * (max_rows - 1)
        df_chunk = df.iloc[start_idx:end_idx]
        df_chunk.to_excel(writer, sheet_name=f"{sheet_name_prefix}_{i+1}", index=False)

In [22]:
df_complete = pd.read_csv('output/complete_neighborhood_data.csv')

In [23]:
df_complete['datetime_update'] = datetime.now()

In [24]:
# colunas = {
#     'grupo_x': 'grupo',
#     'sap_id': 'sap_id',
#     'desc_sap_id': 'desc_item',
#     'cluster_item': 'cluster_item',
#     'rank_item': 'rank_item',
#     'cluster_store_x': 'cluster_store',
#     'modulacao':'modulacao',
#     'store_id':'store_id_string',
#     'nome_store':'desc_loja'
# }
# tipo = {
#     'grupo': str, 
#     'sap_id': int, 
#     'desc_item': str, 
#     'cluster_item': str, 
#     'rank_item': int, 
#     'cluster_store': str, 
#     'modulacao': str, 
#     'store_id_string': str, 
#     'desc_loja': str
# }

In [25]:
project_id = 'methodius-advisory'
dataset = 'dw1_rawtables'
table_name = 'raw_sales_immobiliare'

In [26]:
schema = [
    bigquery.SchemaField('project_id', 'INT64', mode='NULLABLE'),
    bigquery.SchemaField('ads_title', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('link', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('image', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('title', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('price', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('surface', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('bathrooms', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('floor', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('rooms', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('page', 'INT64', mode='NULLABLE'),
    bigquery.SchemaField('neighborhood_link', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('neighborhood', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('drill_down', 'STRING', mode='NULLABLE'),
    bigquery.SchemaField('datetime_update', 'DATETIME', mode='NULLABLE')
]


In [27]:
table_ref = client.dataset(dataset).table(table_name)
try:
    table = client.get_table(table_ref)
    print('A tabela já existe. Os novos dados serão carregados na tabela.')
except:
    print('A tabela não existe. Será criada agora.')
    table = bigquery.Table(table_ref)
    table = client.create_table(table)
    print('A tabela foi criada com sucesso.')

# Carrega os novos dados na tabela existente
job_config = bigquery.LoadJobConfig(write_disposition='WRITE_APPEND', schema=schema)
job = client.load_table_from_dataframe(df_complete.drop_duplicates(), table_ref, job_config=job_config)
job.result()
print('Os novos dados foram carregados com sucesso na tabela existente.')

A tabela já existe. Os novos dados serão carregados na tabela.
Os novos dados foram carregados com sucesso na tabela existente.
